### *Code for P1*

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36 Edg/137.0.0.0"
}
response = requests.get("https://coinmarketcap.com/", headers=headers)
if response.ok:
    print("OK")
else:
    print("Error")
soup = BeautifulSoup(response.content, "html.parser")
coin_table = soup.find("table", {"class": "cmc-table"})
coin_names = []
for row in coin_table.tbody.find_all("tr")[:50]:
    cols = row.find_all("td")
    coin_name = cols[2].text.strip()
    if not ("USD" in coin_name or "DAI" in coin_name):
        coin_names.append(coin_name)
print(coin_names)
def name2symbol(name):
    if name.upper() == name:
        return name[-3:] if "TRUMP" not in name else "TRUMP"
    else:
        for i in range(len(name) - 1, -1, -1):
            if name[i] < 'A' or name[i] > 'Z':
                break
        return name[i + 1:]

coin_symbols = []        
for i in range(len(coin_names)):
    coin_symbols.append(name2symbol(coin_names[i]))
print(coin_symbols)
PROBLEM1_OUTPUT = coin_symbols

### *P1 Output:*
['BTC', 'ETH', 'XRP', 'BNB', 'SOL', 'TRX', 'DOGE', 'ADA', 'HYPE', 'BCH', 'SUI', 'LINK', 'LEO', 'XLM', 'AVAX', 'SHIB', 'TON', 'HBAR', 'LTC', 'XMR', 'DOT', 'BGB', 'UNI', 'AAVE', 'PEPE', 'PI', 'OKB', 'TAO', 'APT', 'NEAR', 'ICP', 'ETC', 'CRO', 'ONDO', 'KAS', 'POL', 'GT', 'MNT', 'BONK', 'VET', 'TRUMP', 'SKY', 'ARB', 'RENDER', 'ENA']

### *Code for P2*

In [ ]:
OKX_BASE_URL = "https://www.okx.com"
PUBLIC_API_ENDPOINTS = {
    "instruments": "/api/v5/public/instruments",
    "candles": "/api/v5/market/candles",
    "funding_rate": "/api/v5/public/funding-rate-history",
}

# Fetch spot instrument data
params = {"instType": "SPOT"}
url = OKX_BASE_URL + PUBLIC_API_ENDPOINTS["instruments"]
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
data = response.json()
assert (data["code"] == '0')

# Filter valid spot trading pairs
valid_pairs = {}
count = 0
for instrument in data["data"]:
    baseCcy = instrument["baseCcy"]
    quoteCcy = instrument["quoteCcy"]
    if baseCcy in PROBLEM1_OUTPUT and quoteCcy == "USDT":
        valid_pairs[baseCcy] = {"spot": f"{baseCcy}-USDT", "swap": f"{baseCcy}-USDT-SWAP"}
        count += 1

# Fetch swap instrument data
params = {"instType": "SWAP"}
url = OKX_BASE_URL + PUBLIC_API_ENDPOINTS["instruments"]
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
data = response.json()

# Filter valid swap contracts
for instrument in data["data"]:
    baseCcy = instrument["baseCcy"]
    quoteCcy = instrument["quoteCcy"]
    if baseCcy in PROBLEM1_OUTPUT and quoteCcy == "USDT":
        valid_pairs[baseCcy] = {"spot": f"{baseCcy}-USDT", "swap": f"{baseCcy}-USDT-SWAP"}
        count += 1

from datetime import datetime, timedelta
import time

def fetch_ohlcv(inst_id, bar, symbol, data_type, limit=100):
    """Fetch OHLCV (Open, High, Low, Close, Volume) data for given instrument"""
    print(f"  Downloading {data_type} OHLCV for {inst_id}")

    url = OKX_BASE_URL + PUBLIC_API_ENDPOINTS["candles"]
    all_data = []

    # Set time range for historical data (1 year)
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=365)

    # Paginate through historical data
    while start_time < end_time:
        params = {
            "instId": inst_id,
            "bar": bar,
            "limit": str(limit),
            "after": str(int(end_time.timestamp() * 1000)),
        }

        try:
            response = requests.get(url, params=params, headers=headers, timeout=20)
            response.raise_for_status()
            data = response.json()

            if data["code"] != "0" or not data["data"]:
                break

            # Parse candle data
            candles = []
            for candle in data["data"]:
                ts, o, h, l, c, vol, vol_ccy, vol_ccy_quote, confirm = candle
                candles.append(
                    {
                        "timestamp": int(ts),
                        "open": float(o),
                        "high": float(h),
                        "low": float(l),
                        "close": float(c),
                        "volume": float(vol),
                    }
                )

            # Update time range for next request
            earliest_ts = min([c["timestamp"] for c in candles])
            end_time = datetime.utcfromtimestamp(earliest_ts / 1000)
            all_data.extend(candles)

            print(f"    Fetched {len(candles)} records, earliest: {end_time}")

            # Respect API rate limits
            time.sleep(0.1)

        except Exception as e:
            print(f"    Error fetching OHLCV: {e}")
            break

    # Save data to CSV
    if all_data:
        df = pd.DataFrame(all_data)
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
        df = df.sort_values("datetime")

        filename = f"historical_data\\{symbol}_{data_type}_ohlcv.csv"
        df.to_csv(filename, index=False)
        print(f"    Saved {len(df)} records to {filename}")
        return filename

    return None

def fetch_funding_rate(inst_id, symbol, limit=100):
    """Fetch funding rate history for perpetual swap contracts"""
    print(f"  Downloading funding rate for {inst_id}")

    url = OKX_BASE_URL + PUBLIC_API_ENDPOINTS["funding_rate"]
    all_data = []
    after = None

    # Paginate through funding rate history
    while True:
        params = {"instId": inst_id, "limit": str(limit)}
        if after:
            params["after"] = after

        try:
            response = requests.get(url, params=params, headers=headers, timeout=15)
            response.raise_for_status()
            data = response.json()

            if data["code"] != "0" or not data["data"]:
                break

            # Parse funding rate data
            for rate in data["data"]:
                all_data.append(
                    {
                        "timestamp": int(rate["fundingTime"]),
                        "rate": float(rate["fundingRate"]),
                        "instId": rate["instId"],
                    }
                )

            # Update pagination marker
            after = data["data"][-1]["fundingTime"]
            print(f"    Fetched {len(data['data'])} funding records")

            # Respect API rate limits
            time.sleep(0.1)

        except Exception as e:
            print(f"    Error fetching funding rates: {e}")
            break

    # Save data to CSV
    if all_data:
        df = pd.DataFrame(all_data)
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
        df = df.sort_values("datetime")

        filename = f"historical_data\\{symbol}_funding_rate.csv"
        df.to_csv(filename, index=False)
        print(f"    Saved {len(df)} funding records to {filename}")
        return filename

    return None

# Create directory for historical data
import os
os.makedirs("historical_data", exist_ok=True)

# Download data for all valid pairs
results = {}
for symbol, pair in valid_pairs.items():
    spot_data = fetch_ohlcv(
        inst_id=pair["spot"], bar="15m", symbol=symbol, data_type="spot"
    )
    futures_data = fetch_ohlcv(
        inst_id=pair["swap"], bar="15m", symbol=symbol, data_type="swap"
    )
    funding_data = fetch_funding_rate(inst_id=pair["swap"], symbol=symbol)
    
    results[symbol] = {
        "spot": spot_data,
        "futures": futures_data,
        "funding": funding_data,
    }

### *Code for P3*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta


class BackTest:
    def __init__(self):
        self.data_folder = "historical_data"
        self.initial_cash = 1
        self.cash = self.initial_cash
        self.num_hot_coins = 10

        self.coins = self._find_available_coins()
        self.data = self._load_data()

        self.spot_qty = {}
        self.swap_qty = {}
        self.spot_qty_history = pd.DataFrame(index=self.data.index, columns=self.coins)
        self.swap_qty_history = pd.DataFrame(index=self.data.index, columns=self.coins)
        for coin in self.coins:
            self.spot_qty[coin] = 0.0
            self.spot_qty_history[coin] = 0.0
            self.swap_qty[coin] = 0.0
            self.swap_qty_history[coin] = 0.0

        self.earlist_time = datetime.strptime("2025-06-24 08:00:00", "%Y-%m-%d %H:%M:%S")
        self.latest_time = datetime.strptime("2025-07-08 18:00:00", "%Y-%m-%d %H:%M:%S")

        self.threshold = 0.05
        self.ratio = 0.45
        self.total_val = 1e6
        self.time_delta = timedelta(hours=8)

    def sell(self, coin, qty, cur_time, type):
        if (type == "spot" and qty > self.spot_qty[coin]) or (type == "swap" and qty > self.swap_qty[coin]):
            print("Error: Insufficient spot quantity!")

        if type == "spot":
            self.cash += qty * self.data.loc[cur_time, f"{coin}_spot_price"]
            self.spot_qty[coin] -= qty

        elif type == "swap":
            self.cash += qty * self.data.loc[cur_time, f"{coin}_swap_price"]
            self.swap_qty[coin] -= qty
        else:
            print("Error: Invalid type!")

    def buy(self, coin, qty, cur_time, type):
        if (type == "spot" and qty * self.data.loc[cur_time, f"{coin}_spot_price"] > self.cash
            or type == "swap" and qty * self.data.loc[cur_time, f"{coin}_swap_price"] > self.cash):
            print("Error: Insufficient cash!")

        if type == "spot":
            self.cash -= qty * self.data.loc[cur_time, f"{coin}_spot_price"]
            self.spot_qty[coin] += qty

        elif type == "swap":
            self.cash -= qty * self.data.loc[cur_time, f"{coin}_swap_price"]
            self.swap_qty[coin] += qty
        else:
            print("Error: Invalid type!")

    def _find_available_coins(self):
        files = os.listdir(self.data_folder)
        spot_coins = set(f.split("_")[0] for f in files if "_spot_" in f)
        future_coins = set(f.split("_")[0] for f in files if "_swap_" in f)
        funding_coins = set(f.split("_")[0] for f in files if "_funding_" in f)
        return sorted(spot_coins & future_coins & funding_coins)

    def _load_data(self):
        if not self.coins:
            print("Warning: No coins available!")
            return pd.DataFrame()

        all_data = []
        for coin in self.coins:
            try:
                spot_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_spot_ohlcv.csv"),
                    usecols=["datetime", "close"],
                    parse_dates=["datetime"],
                ).rename(columns={"close": f"{coin}_spot_price"})

                future_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_swap_ohlcv.csv"),
                    usecols=["datetime", "close"],
                    parse_dates=["datetime"],
                ).rename(columns={"close": f"{coin}_swap_price"})

                funding_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_funding_rate.csv"),
                    usecols=["datetime", "rate"],
                    parse_dates=["datetime"],
                ).rename(columns={"rate": f"{coin}_funding_rate"})

                merged = spot_df.merge(future_df, on="datetime", how="left")
                merged = merged.merge(funding_df, on="datetime", how="left")
                merged[f"{coin}_funding_rate"] = merged[f"{coin}_funding_rate"].ffill()
                all_data.append(merged)
            except Exception as e:
                print(f"Error loading {coin} data: {e}")

        if not all_data:
            return pd.DataFrame()

        full_data = all_data[0]
        for df in all_data[1:]:
            full_data = full_data.merge(df, on="datetime", how="outer")

        full_data = full_data.sort_values("datetime").set_index("datetime")
        full_time_range = pd.date_range(
            start=full_data.index.min(), end=full_data.index.max(), freq="h"
        )
        full_data = full_data.reindex(full_time_range)
        full_data = full_data.interpolate(method="linear")

        return full_data

    def pre_trading(self, cur_time):
        week_ago = cur_time - timedelta(days=7)
        idx = self.data.index.get_indexer([week_ago], method="nearest")[0]
        week_ago_data = self.data.iloc[idx]
        current_data = self.data.loc[cur_time]

        gains = {}
        for coin in self.coins:
            try:
                start_price = week_ago_data[f"{coin}_spot_price"]
                end_price = current_data[f"{coin}_spot_price"]
                gain = (end_price - start_price) / start_price
                gains[coin] = gain
            except:
                gains[coin] = -999

        sorted_coins = sorted(gains.items(), key=lambda x: x[1], reverse=True)
        hot_coins = [coin for coin, gain in sorted_coins[: self.num_hot_coins]]

        if abs(2 * self.ratio - 1) > self.threshold:
            self.ratio = 0.5 + self.threshold if self.ratio > 0.5 else 0.5 - self.threshold

        return hot_coins

    def trading(self, cur_time):
        hot_coins = self.pre_trading(cur_time)

        for coin in self.coins:
            self.sell(coin, self.spot_qty[coin], cur_time, "spot")
            self.sell(coin, self.swap_qty[coin], cur_time, "swap")

        for coin in hot_coins:
            spot_price = self.data.loc[cur_time, f"{coin}_spot_price"]
            swap_price = self.data.loc[cur_time, f"{coin}_swap_price"]

            spot_qty = self.cash * self.ratio / self.num_hot_coins / spot_price
            swap_qty = self.cash * (1 - self.ratio) / self.num_hot_coins / swap_price

            if spot_qty > self.spot_qty[coin]:
                self.buy(coin, spot_qty - self.spot_qty[coin], cur_time, "spot")
            else:
                self.sell(coin, self.spot_qty[coin] - spot_qty, cur_time, "spot")

            if swap_qty > self.swap_qty[coin]:
                self.buy(coin, swap_qty - self.swap_qty[coin], cur_time, "swap")
            else:
                self.sell(coin, self.swap_qty[coin] - swap_qty, cur_time, "swap")

    def update_state(self, cur_time):
        next_time = cur_time + self.time_delta
        self.total_val = self.cash + sum(
            self.spot_qty[coin] * self.data.loc[next_time, f"{coin}_spot_price"]
            + self.swap_qty[coin] * self.data.loc[next_time, f"{coin}_swap_price"]
            + self.data.loc[next_time, f"{coin}_funding_rate"] * self.swap_qty[coin]
            for coin in self.coins
        )
        for coin in self.coins:
            self.spot_qty_history.loc[cur_time:next_time, coin] = self.spot_qty[coin]
            self.swap_qty_history.loc[cur_time:next_time, coin] = self.swap_qty[coin]
        return next_time

    def analyze(self):
        self.portfolio_history = pd.DataFrame(index=self.data.index)
        self.portfolio_history["spot_value"] = 0.0
        self.portfolio_history["futures_value"] = 0.0
        self.portfolio_history["funding_value"] = 0.0
        self.portfolio_history["total_value"] = 0.0

        for coin in self.coins:
            self.portfolio_history["spot_value"] += (
                self.spot_qty_history[coin] * self.data[f"{coin}_spot_price"]
            )
            self.portfolio_history["futures_value"] += (
                self.swap_qty_history[coin] * self.data[f"{coin}_swap_price"]
            )
            self.portfolio_history["funding_value"] += (
                self.swap_qty_history[coin] * self.data[f"{coin}_funding_rate"]
            )

        print(self.spot_qty_history["SHIB"])

        self.portfolio_history["total_value"] = (
            self.portfolio_history["spot_value"]
            + self.portfolio_history["futures_value"]
            + self.portfolio_history["funding_value"]
            + self.cash
        )

        daily_returns = self.portfolio_history["total_value"].pct_change().dropna()

        self.metrics = {
            "Annualized Return": (1 + daily_returns.mean()) ** 365 - 1,
            "Sharpe Ratio": daily_returns.mean() / daily_returns.std() * np.sqrt(365),
            "Max Drawdown": (
                1
                - self.portfolio_history["total_value"]
                / self.portfolio_history["total_value"].cummax()
            ).max(),
        }
        self.metrics["Calmar Ratio"] = (
            self.metrics["Annualized Return"] / self.metrics["Max Drawdown"]
        )

        print("\nPerformance Metrics:")
        for metric, value in self.metrics.items():
            print(f"{metric}: {value:.4f}")

    def visualize(self):
        plt.figure(figsize=(15, 10))

        plt.subplot(2, 1, 1)
        plt.plot(self.portfolio_history["spot_value"], label="Spot PnL")
        plt.plot(self.portfolio_history["futures_value"], label="Futures PnL")
        plt.plot(self.portfolio_history["funding_value"], label="Funding PnL")
        plt.title("Component PnL Curves")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)

        plt.subplot(2, 1, 2)
        plt.plot(self.portfolio_history["total_value"], label="Total Portfolio Value")
        plt.title("Total Portfolio Value Over Time")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.grid(True)

        plt.show()

    def run_strategy(self):
        cur_time = self.earlist_time
        while cur_time <= self.latest_time - self.time_delta:
            self.trading(cur_time)
            cur_time = self.update_state(cur_time)

        self.analyze()
        self.visualize()


if __name__ == "__main__":
    bt = BackTest()
    bt.run_strategy()

### *P3 Performance Metrics*:
Annualized Return: -0.7888

Sharpe Ratio: -0.4312

Max Drawdown: 1.4595

Calmar Ratio: -0.5404

### *Code for P4*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta


class EnhancedBackTest:
    def __init__(self):
        # Initialize configuration parameters
        self.data_folder = "historical_data"
        self.vol_window = 7 * 24 * 4
        self.initial_cash = 1
        self.cash = self.initial_cash
        self.num_hot_coins = 10
        self.bid_ask = 0.0007
        self.slippage = 0.0005

        # Initialize data structures
        self.coins = self._find_available_coins()
        self.data = self._load_data()

        # Initialize portfolio tracking
        self.spot_qty = {}
        self.swap_qty = {}
        self.spot_qty_history = pd.DataFrame(index=self.data.index, columns=self.coins)
        self.swap_qty_history = pd.DataFrame(index=self.data.index, columns=self.coins)
        for coin in self.coins:
            self.spot_qty[coin] = 0.0
            self.spot_qty_history[coin] = 0.0
            self.swap_qty[coin] = 0.0
            self.swap_qty_history[coin] = 0.0

        # Set time parameters
        self.earlist_time = datetime.strptime("2025-06-24 08:00:00", "%Y-%m-%d %H:%M:%S")
        self.latest_time = datetime.strptime("2025-07-08 18:00:00", "%Y-%m-%d %H:%M:%S")

        # Initialize strategy parameters
        self.base_threshold = 0.05
        self.threshold = self.base_threshold
        self.ratio = 0.45
        self.total_val = 1e6
        self.time_delta = timedelta(hours=8)

    def sell(self, coin, qty, cur_time, type):
        # Validate position quantity
        if (type == "spot" and qty > self.spot_qty[coin]) or (type == "swap" and qty > self.swap_qty[coin]):
            print("Error: Insufficient spot quantity!")

        # Execute spot sell order
        if type == "spot":
            self.cash += qty * self.data.loc[cur_time, f"{coin}_spot_price"]
            self.cash -= self._txn_cost(qty, self.data.loc[cur_time, f"{coin}_spot_price"])
            self.spot_qty[coin] -= qty

        # Execute futures sell order
        elif type == "swap":
            self.cash += qty * self.data.loc[cur_time, f"{coin}_swap_price"]
            self.cash -= self._txn_cost(qty, self.data.loc[cur_time, f"{coin}_swap_price"])
            self.swap_qty[coin] -= qty
        else:
            print("Error: Invalid type!")

    def buy(self, coin, qty, cur_time, type):
        # Validate available cash
        if (type == "spot" and qty * self.data.loc[cur_time, f"{coin}_spot_price"] > self.cash) or (type == "swap" and qty * self.data.loc[cur_time, f"{coin}_swap_price"] > self.cash):
            print("Error: Insufficient cash!")

        # Execute spot buy order
        if type == "spot":
            self.cash -= qty * self.data.loc[cur_time, f"{coin}_spot_price"]
            self.cash -= self._txn_cost(qty, self.data.loc[cur_time, f"{coin}_spot_price"])
            self.spot_qty[coin] += qty

        # Execute futures buy order
        elif type == "swap":
            self.cash -= qty * self.data.loc[cur_time, f"{coin}_swap_price"]
            self.cash -= self._txn_cost(qty, self.data.loc[cur_time, f"{coin}_swap_price"])
            self.swap_qty[coin] += qty
        else:
            print("Error: Invalid type!")

    def _find_available_coins(self):
        # Identify coins with complete data across all required markets
        files = os.listdir(self.data_folder)
        spot_coins = set(f.split("_")[0] for f in files if "_spot_" in f)
        future_coins = set(f.split("_")[0] for f in files if "_swap_" in f)
        funding_coins = set(f.split("_")[0] for f in files if "_funding_" in f)
        return sorted(spot_coins & future_coins & funding_coins)

    def _load_data(self):
        # Load and merge market data for all available coins
        if not self.coins:
            print("Warning: No coins available!")
            return pd.DataFrame()

        all_data = []
        for coin in self.coins:
            try:
                # Load and merge spot, futures and funding data
                spot_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_spot_ohlcv.csv"),
                    usecols=["datetime", "close"],
                    parse_dates=["datetime"],
                ).rename(columns={"close": f"{coin}_spot_price"})

                future_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_swap_ohlcv.csv"),
                    usecols=["datetime", "close"],
                    parse_dates=["datetime"],
                ).rename(columns={"close": f"{coin}_swap_price"})

                funding_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_funding_rate.csv"),
                    usecols=["datetime", "rate"],
                    parse_dates=["datetime"],
                ).rename(columns={"rate": f"{coin}_funding_rate"})

                merged = spot_df.merge(future_df, on="datetime", how="left")
                merged = merged.merge(funding_df, on="datetime", how="left")
                merged[f"{coin}_funding_rate"] = merged[f"{coin}_funding_rate"].ffill()
                all_data.append(merged)
            except Exception as e:
                print(f"Error loading {coin} data: {e}")

        if not all_data:
            return pd.DataFrame()

        # Combine all coin data into single DataFrame
        full_data = all_data[0]
        for df in all_data[1:]:
            full_data = full_data.merge(df, on="datetime", how="outer")

        # Process and clean time series data
        full_data = full_data.sort_values("datetime").set_index("datetime")
        full_time_range = pd.date_range(start=full_data.index.min(), end=full_data.index.max(), freq="h")
        full_data = full_data.reindex(full_time_range)
        full_data = full_data.interpolate(method="linear")

        return full_data

    def _txn_cost(self, qty, price):
        # Calculate total transaction costs
        return qty * price * (self.bid_ask + self.slippage)

    def _volatility_scaling(self, cur_time):
        # Calculate market volatility and adjust threshold
        vol_all = []
        for coin in self.coins:
            prices = self.data[f"{coin}_spot_price"]
            returns = prices.pct_change().fillna(0)
            idx = self.data.index.get_loc(cur_time)
            if idx < self.vol_window:
                vol = returns[:idx].std()
            else:
                vol = returns[idx - self.vol_window : idx].std()
            vol = vol * np.sqrt(365 * 24)
            vol_all.append(vol)

        avg_vol = np.mean([v for v in vol_all if not np.isnan(v)])
        return self.base_threshold * (avg_vol / 0.8)

    def _volatility_target_weights(self, cur_time, hot_coins):
        # Calculate inverse volatility weights for portfolio allocation
        vols = []
        for coin in hot_coins:
            prices = self.data[f"{coin}_spot_price"]
            returns = prices.pct_change().fillna(0)
            idx = self.data.index.get_loc(cur_time)
            if idx < self.vol_window:
                vol = returns[:idx].std()
            else:
                vol = returns[idx - self.vol_window : idx].std()
            vols.append(vol)

        inv_vols = [1 / v if v > 0 else 0 for v in vols]
        total = sum(inv_vols)
        weights = [v / total for v in inv_vols]
        return dict(zip(hot_coins, weights))

    def pre_trading(self, cur_time):
        # Calculate weekly returns and select top performing coins
        week_ago = cur_time - timedelta(days=7)
        idx = self.data.index.get_indexer([week_ago], method="nearest")[0]
        week_ago_data = self.data.iloc[idx]
        current_data = self.data.loc[cur_time]

        gains = {}
        for coin in self.coins:
            try:
                start_price = week_ago_data[f"{coin}_spot_price"]
                end_price = current_data[f"{coin}_spot_price"]
                gain = (end_price - start_price) / start_price
                gains[coin] = gain
            except:
                gains[coin] = -999

        sorted_coins = sorted(gains.items(), key=lambda x: x[1], reverse=True)
        hot_coins = [coin for coin, gain in sorted_coins[: self.num_hot_coins]]

        # Update volatility-adjusted parameters
        self.threshold = self._volatility_scaling(cur_time)
        weights = self._volatility_target_weights(cur_time, hot_coins)

        # Adjust spot/futures ratio if beyond threshold
        if abs(2 * self.ratio - 1) > self.threshold:
            self.ratio = 0.5 + self.threshold if self.ratio > 0.5 else 0.5 - self.threshold

        return hot_coins, weights

    def trading(self, cur_time):
        # Get current trading signals and weights
        hot_coins, weights = self.pre_trading(cur_time)

        # Liquidate all existing positions
        for coin in self.coins:
            self.sell(coin, self.spot_qty[coin], cur_time, "spot")
            self.sell(coin, self.swap_qty[coin], cur_time, "swap")

        # Rebalance portfolio according to new signals
        for coin in hot_coins:
            spot_price = self.data.loc[cur_time, f"{coin}_spot_price"]
            swap_price = self.data.loc[cur_time, f"{coin}_swap_price"]

            # Calculate target position sizes
            spot_qty = self.cash * self.ratio * weights[coin] / self.num_hot_coins / spot_price
            swap_qty = self.cash * (1 - self.ratio) * weights[coin] / self.num_hot_coins / swap_price

            # Execute rebalancing trades
            if spot_qty > self.spot_qty[coin]:
                self.buy(coin, spot_qty - self.spot_qty[coin], cur_time, "spot")
            else:
                self.sell(coin, self.spot_qty[coin] - spot_qty, cur_time, "spot")

            if swap_qty > self.swap_qty[coin]:
                self.buy(coin, swap_qty - self.swap_qty[coin], cur_time, "swap")
            else:
                self.sell(coin, self.swap_qty[coin] - swap_qty, cur_time, "swap")

    def update_state(self, cur_time):
        # Update portfolio valuation and move to next time period
        next_time = cur_time + self.time_delta
        self.total_val = self.cash + sum(
            self.spot_qty[coin] * self.data.loc[next_time, f"{coin}_spot_price"]
            + self.swap_qty[coin] * self.data.loc[next_time, f"{coin}_swap_price"]
            + self.data.loc[next_time, f"{coin}_funding_rate"] * self.swap_qty[coin]
            for coin in self.coins
        )
        for coin in self.coins:
            self.spot_qty_history.loc[cur_time:next_time, coin] = self.spot_qty[coin]
            self.swap_qty_history.loc[cur_time:next_time, coin] = self.swap_qty[coin]
        return next_time

    def analyze(self):
        # Initialize portfolio history DataFrame
        self.portfolio_history = pd.DataFrame(index=self.data.index)
        self.portfolio_history["spot_value"] = 0.0
        self.portfolio_history["futures_value"] = 0.0
        self.portfolio_history["funding_value"] = 0.0
        self.portfolio_history["total_value"] = self.cash

        # Calculate component values over time
        for coin in self.coins:
            self.portfolio_history["spot_value"] += self.spot_qty_history[coin] * self.data[f"{coin}_spot_price"]
            self.portfolio_history["futures_value"] += self.swap_qty_history[coin] * self.data[f"{coin}_swap_price"]
            self.portfolio_history["funding_value"] += self.swap_qty_history[coin] * self.data[f"{coin}_funding_rate"]

        # Calculate total portfolio value
        self.portfolio_history["total_value"] = (
            self.portfolio_history["spot_value"]
            + self.portfolio_history["futures_value"]
            + self.portfolio_history["funding_value"]
            + self.cash
        )

        # Calculate performance metrics
        daily_returns = self.portfolio_history["total_value"].pct_change().dropna()

        self.metrics = {
            "Annualized Return": (1 + daily_returns.mean()) ** 365 - 1,
            "Sharpe Ratio": daily_returns.mean() / daily_returns.std() * np.sqrt(365),
            "Max Drawdown": (1 - self.portfolio_history["total_value"] / self.portfolio_history["total_value"].cummax()).max(),
        }
        self.metrics["Calmar Ratio"] = self.metrics["Annualized Return"] / self.metrics["Max Drawdown"]

        # Display performance results
        print("\nPerformance Metrics:")
        for metric, value in self.metrics.items():
            print(f"{metric}: {value:.4f}")

    def visualize(self):
        # Create performance visualization plots
        plt.figure(figsize=(15, 10))

        # Component performance plot
        plt.subplot(2, 1, 1)
        plt.plot(self.portfolio_history["spot_value"], label="Spot PnL")
        plt.plot(self.portfolio_history["futures_value"], label="Futures PnL")
        plt.plot(self.portfolio_history["funding_value"], label="Funding PnL")
        plt.title("Component PnL Curves")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)

        # Total portfolio plot
        plt.subplot(2, 1, 2)
        plt.plot(self.portfolio_history["total_value"], label="Total Portfolio Value")
        plt.title("Total Portfolio Value Over Time")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.grid(True)

        plt.show()

    def run_strategy(self):
        # Execute backtest over specified time period
        cur_time = self.earlist_time
        while cur_time <= self.latest_time - self.time_delta:
            self.trading(cur_time)
            cur_time = self.update_state(cur_time)

        # Analyze and visualize results
        self.analyze()
        self.visualize()


if __name__ == "__main__":
    bt = EnhancedBackTest()
    bt.run_strategy()

### *P4 Performance Metrics*:

Annualized Return: -0.0184

Sharpe Ratio: -0.0718

Max Drawdown: 0.1756

Calmar Ratio: -0.1050

### *Code for P5 (pending revision)*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta

# === New: torch & MLP imports ===
import torch
import torch.nn as nn
import torch.optim as optim


class MLPWeights(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=-1),
        )

    def forward(self, x):
        return self.net(x)


class EnhancedBackTest:
    def __init__(self):
        # Initialize configuration parameters
        self.data_folder = "historical_data"
        self.vol_window = 7 * 24 * 4
        self.initial_cash = 1
        self.cash = self.initial_cash
        self.num_hot_coins = 10
        self.bid_ask = 0.0007
        self.slippage = 0.0005

        # Initialize data structures
        self.coins = self._find_available_coins()
        self.data = self._load_data()

        # Initialize portfolio tracking
        self.spot_qty = {}
        self.swap_qty = {}
        self.spot_qty_history = pd.DataFrame(index=self.data.index, columns=self.coins)
        self.swap_qty_history = pd.DataFrame(index=self.data.index, columns=self.coins)
        for coin in self.coins:
            self.spot_qty[coin] = 0.0
            self.spot_qty_history[coin] = 0.0
            self.swap_qty[coin] = 0.0
            self.swap_qty_history[coin] = 0.0

        # Set time parameters
        self.earlist_time = datetime.strptime(
            "2025-06-24 08:00:00", "%Y-%m-%d %H:%M:%S"
        )
        self.latest_time = datetime.strptime("2025-07-08 18:00:00", "%Y-%m-%d %H:%M:%S")

        # Initialize strategy parameters
        self.base_threshold = 0.05
        self.threshold = self.base_threshold
        self.ratio = 0.45
        self.total_val = 1
        self.time_delta = timedelta(hours=8)

        # === New: MLP for weights learning ===
        self.mlp_input_dim = (
            self.num_hot_coins * 2
        )  # e.g., use price returns + volatility per coin
        self.mlp_hidden_dim = 64
        self.mlp = MLPWeights(
            self.mlp_input_dim, self.mlp_hidden_dim, self.num_hot_coins
        )
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.mlp.to(self.device)

    def sell(self, coin, qty, cur_time, type):
        # Validate position quantity
        if (type == "spot" and qty > self.spot_qty[coin]) or (
            type == "swap" and qty > self.swap_qty[coin]
        ):
            print("Error: Insufficient spot quantity!")

        # Execute spot sell order
        if type == "spot":
            self.cash += qty * self.data.loc[cur_time, f"{coin}_spot_price"]
            self.cash -= self._txn_cost(
                qty, self.data.loc[cur_time, f"{coin}_spot_price"]
            )
            self.spot_qty[coin] -= qty

        # Execute futures sell order
        elif type == "swap":
            self.cash += qty * self.data.loc[cur_time, f"{coin}_swap_price"]
            self.cash -= self._txn_cost(
                qty, self.data.loc[cur_time, f"{coin}_swap_price"]
            )
            self.swap_qty[coin] -= qty
        else:
            print("Error: Invalid type!")

    def buy(self, coin, qty, cur_time, type):
        # Validate available cash
        if (
            type == "spot"
            and qty * self.data.loc[cur_time, f"{coin}_spot_price"] > self.cash
        ) or (
            type == "swap"
            and qty * self.data.loc[cur_time, f"{coin}_swap_price"] > self.cash
        ):
            print("Error: Insufficient cash!")

        # Execute spot buy order
        if type == "spot":
            self.cash -= qty * self.data.loc[cur_time, f"{coin}_spot_price"]
            self.cash -= self._txn_cost(
                qty, self.data.loc[cur_time, f"{coin}_spot_price"]
            )
            self.spot_qty[coin] += qty

        # Execute futures buy order
        elif type == "swap":
            self.cash -= qty * self.data.loc[cur_time, f"{coin}_swap_price"]
            self.cash -= self._txn_cost(
                qty, self.data.loc[cur_time, f"{coin}_swap_price"]
            )
            self.swap_qty[coin] += qty
        else:
            print("Error: Invalid type!")

    def _find_available_coins(self):
        # Identify coins with complete data across all required markets
        files = os.listdir(self.data_folder)
        spot_coins = set(f.split("_")[0] for f in files if "_spot_" in f)
        future_coins = set(f.split("_")[0] for f in files if "_swap_" in f)
        funding_coins = set(f.split("_")[0] for f in files if "_funding_" in f)
        return sorted(spot_coins & future_coins & funding_coins)

    def _load_data(self):
        # Load and merge market data for all available coins
        if not self.coins:
            print("Warning: No coins available!")
            return pd.DataFrame()

        all_data = []
        for coin in self.coins:
            try:
                # Load and merge spot, futures and funding data
                spot_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_spot_ohlcv.csv"),
                    usecols=["datetime", "close"],
                    parse_dates=["datetime"],
                ).rename(columns={"close": f"{coin}_spot_price"})

                future_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_swap_ohlcv.csv"),
                    usecols=["datetime", "close"],
                    parse_dates=["datetime"],
                ).rename(columns={"close": f"{coin}_swap_price"})

                funding_df = pd.read_csv(
                    os.path.join(self.data_folder, f"{coin}_funding_rate.csv"),
                    usecols=["datetime", "rate"],
                    parse_dates=["datetime"],
                ).rename(columns={"rate": f"{coin}_funding_rate"})

                merged = spot_df.merge(future_df, on="datetime", how="left")
                merged = merged.merge(funding_df, on="datetime", how="left")
                merged[f"{coin}_funding_rate"] = merged[f"{coin}_funding_rate"].ffill()
                all_data.append(merged)
            except Exception as e:
                print(f"Error loading {coin} data: {e}")

        if not all_data:
            return pd.DataFrame()

        # Combine all coin data into single DataFrame
        full_data = all_data[0]
        for df in all_data[1:]:
            full_data = full_data.merge(df, on="datetime", how="outer")

        # Process and clean time series data
        full_data = full_data.sort_values("datetime").set_index("datetime")
        full_time_range = pd.date_range(
            start=full_data.index.min(), end=full_data.index.max(), freq="h"
        )
        full_data = full_data.reindex(full_time_range)
        full_data = full_data.interpolate(method="linear")

        return full_data

    def _txn_cost(self, qty, price):
        # Calculate total transaction costs
        return qty * price * (self.bid_ask + self.slippage)

    def _volatility_scaling(self, cur_time):
        # Calculate market volatility and adjust threshold
        vol_all = []
        for coin in self.coins:
            prices = self.data[f"{coin}_spot_price"]
            returns = prices.pct_change().fillna(0)
            idx = self.data.index.get_loc(cur_time)
            if idx < self.vol_window:
                vol = returns[:idx].std()
            else:
                vol = returns[idx - self.vol_window : idx].std()
            vol = vol * np.sqrt(365 * 24)
            vol_all.append(vol)

        avg_vol = np.mean([v for v in vol_all if not np.isnan(v)])
        return self.base_threshold * (avg_vol / 0.8)

    def _volatility_target_weights(
        self, cur_time, hot_coins, mlp_weights=None, mlp_features=None
    ):
        # Use MLP if weights are provided, else default to inverse vol
        if mlp_weights is not None:
            weights = mlp_weights
            weights = weights / weights.sum()  # Ensure normalization
            return dict(zip(hot_coins, weights.tolist()))
        # Default inverse volatility allocation (for baseline)
        vols = []
        for coin in hot_coins:
            prices = self.data[f"{coin}_spot_price"]
            returns = prices.pct_change().fillna(0)
            idx = self.data.index.get_loc(cur_time)
            if idx < self.vol_window:
                vol = returns[:idx].std()
            else:
                vol = returns[idx - self.vol_window : idx].std()
            vols.append(vol)
        inv_vols = [1 / v if v > 0 else 0 for v in vols]
        total = sum(inv_vols)
        weights = [v / total for v in inv_vols]
        return dict(zip(hot_coins, weights))

    def get_mlp_features(self, cur_time, hot_coins):
        # Example feature: 7d returns + 7d stddev for each hot coin
        idx = self.data.index.get_loc(cur_time)
        feature_list = []
        for coin in hot_coins:
            prices = self.data[f"{coin}_spot_price"]
            returns = prices.pct_change().fillna(0)
            if idx < 7 * 24:
                past_returns = returns[:idx]
            else:
                past_returns = returns[idx - 7 * 24 : idx]
            # 7d return
            ret = (prices.iloc[idx] - prices.iloc[max(0, idx - 7 * 24)]) / (
                prices.iloc[max(0, idx - 7 * 24)] + 1e-9
            )
            vol = past_returns.std()
            feature_list.append(ret)
            feature_list.append(vol)
        return np.array(feature_list, dtype=np.float32)

    def pre_trading(self, cur_time, mlp_infer=True):
        # Calculate weekly returns and select top performing coins
        week_ago = cur_time - timedelta(days=7)
        idx = self.data.index.get_indexer([week_ago], method="nearest")[0]
        week_ago_data = self.data.iloc[idx]
        current_data = self.data.loc[cur_time]

        gains = {}
        for coin in self.coins:
            try:
                start_price = week_ago_data[f"{coin}_spot_price"]
                end_price = current_data[f"{coin}_spot_price"]
                gain = (end_price - start_price) / (start_price + 1e-9)
                gains[coin] = gain
            except:
                gains[coin] = -999

        sorted_coins = sorted(gains.items(), key=lambda x: x[1], reverse=True)
        hot_coins = [coin for coin, gain in sorted_coins[: self.num_hot_coins]]

        # Update volatility-adjusted parameters
        self.threshold = self._volatility_scaling(cur_time)

        # === New: MLP-based weights ===
        mlp_features = self.get_mlp_features(cur_time, hot_coins)
        if mlp_infer:
            with torch.no_grad():
                inp = torch.tensor(
                    mlp_features, dtype=torch.float32, device=self.device
                ).unsqueeze(0)
                weights = self.mlp(inp).squeeze(0).cpu().numpy()
        else:
            weights = None  # For training, return features

        # Adjust spot/futures ratio if beyond threshold
        if abs(2 * self.ratio - 1) > self.threshold:
            self.ratio = (
                0.5 + self.threshold if self.ratio > 0.5 else 0.5 - self.threshold
            )

        return hot_coins, weights, mlp_features

    def trading(self, cur_time, weights=None):
        # Get current trading signals and weights
        hot_coins, mlp_weights, mlp_features = self.pre_trading(
            cur_time, mlp_infer=(weights is None)
        )
        if weights is not None:
            mlp_weights = weights
        weights_dict = self._volatility_target_weights(
            cur_time, hot_coins, mlp_weights, mlp_features
        )

        # Liquidate all existing positions
        for coin in self.coins:
            self.sell(coin, self.spot_qty[coin], cur_time, "spot")
            self.sell(coin, self.swap_qty[coin], cur_time, "swap")

        # Rebalance portfolio according to new signals
        for i, coin in enumerate(hot_coins):
            spot_price = self.data.loc[cur_time, f"{coin}_spot_price"]
            swap_price = self.data.loc[cur_time, f"{coin}_swap_price"]
            # Calculate target position sizes
            spot_qty = self.cash * self.ratio * weights_dict[coin] / spot_price
            swap_qty = self.cash * (1 - self.ratio) * weights_dict[coin] / swap_price
            # Execute rebalancing trades
            if spot_qty > self.spot_qty[coin]:
                self.buy(coin, spot_qty - self.spot_qty[coin], cur_time, "spot")
            else:
                self.sell(coin, self.spot_qty[coin] - spot_qty, cur_time, "spot")
            if swap_qty > self.swap_qty[coin]:
                self.buy(coin, swap_qty - self.swap_qty[coin], cur_time, "swap")
            else:
                self.sell(coin, self.swap_qty[coin] - swap_qty, cur_time, "swap")

        return hot_coins, mlp_weights, mlp_features

    def update_state(self, cur_time):
        # Update portfolio valuation and move to next time period
        next_time = cur_time + self.time_delta
        self.total_val = self.cash + sum(
            self.spot_qty[coin] * self.data.loc[next_time, f"{coin}_spot_price"]
            + self.swap_qty[coin] * self.data.loc[next_time, f"{coin}_swap_price"]
            + self.data.loc[next_time, f"{coin}_funding_rate"] * self.swap_qty[coin]
            for coin in self.coins
        )
        for coin in self.coins:
            self.spot_qty_history.loc[cur_time:next_time, coin] = self.spot_qty[coin]
            self.swap_qty_history.loc[cur_time:next_time, coin] = self.swap_qty[coin]
        return next_time

    def analyze(self):
        # Initialize portfolio history DataFrame
        self.portfolio_history = pd.DataFrame(index=self.data.index)
        self.portfolio_history["spot_value"] = 0.0
        self.portfolio_history["futures_value"] = 0.0
        self.portfolio_history["funding_value"] = 0.0
        self.portfolio_history["total_value"] = self.cash

        # Calculate component values over time
        for coin in self.coins:
            self.portfolio_history["spot_value"] += (
                self.spot_qty_history[coin] * self.data[f"{coin}_spot_price"]
            )
            self.portfolio_history["futures_value"] += (
                self.swap_qty_history[coin] * self.data[f"{coin}_swap_price"]
            )
            self.portfolio_history["funding_value"] += (
                self.swap_qty_history[coin] * self.data[f"{coin}_funding_rate"]
            )

        # Calculate total portfolio value
        self.portfolio_history["total_value"] = (
            self.portfolio_history["spot_value"]
            + self.portfolio_history["futures_value"]
            + self.portfolio_history["funding_value"]
            + self.cash
        )

        # Calculate performance metrics
        daily_returns = self.portfolio_history["total_value"].pct_change().dropna()

        self.metrics = {
            "Annualized Return": (1 + daily_returns.mean()) ** 365 - 1,
            "Sharpe Ratio": daily_returns.mean() / daily_returns.std() * np.sqrt(365),
            "Max Drawdown": (
                1
                - self.portfolio_history["total_value"]
                / self.portfolio_history["total_value"].cummax()
            ).max(),
        }
        self.metrics["Calmar Ratio"] = (
            self.metrics["Annualized Return"] / self.metrics["Max Drawdown"]
        )

        # Display performance results
        print("\nPerformance Metrics:")
        for metric, value in self.metrics.items():
            print(f"{metric}: {value:.4f}")

    def visualize(self):
        # Create performance visualization plots
        plt.figure(figsize=(15, 10))
        plt.subplot(2, 1, 1)
        plt.plot(self.portfolio_history["spot_value"], label="Spot PnL")
        plt.plot(self.portfolio_history["futures_value"], label="Futures PnL")
        plt.plot(self.portfolio_history["funding_value"], label="Funding PnL")
        plt.title("Component PnL Curves")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.subplot(2, 1, 2)
        plt.plot(self.portfolio_history["total_value"], label="Total Portfolio Value")
        plt.title("Total Portfolio Value Over Time")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.grid(True)
        plt.show()

    def build_trainset(self, window_size=30, step_size=1):
        # 构建滚动窗口的训练集
        train_X, train_y = [], []
        date_idx = self.data.index
        for i in range(window_size, len(date_idx) - 1, step_size):
            cur_time = date_idx[i]
            hot_coins, _, features = self.pre_trading(cur_time, mlp_infer=False)
            # 生成目标y: 下一周期涨幅排名靠前的币的 one-hot
            next_time = date_idx[i + 1]
            next_gains = []
            for coin in hot_coins:
                try:
                    start = self.data.loc[cur_time, f"{coin}_spot_price"]
                    end = self.data.loc[next_time, f"{coin}_spot_price"]
                    gain = (end - start) / (start + 1e-9)
                except:
                    gain = -999
                next_gains.append(gain)
            onehot = np.zeros(len(next_gains), dtype=np.float32)
            if np.max(next_gains) > -998:
                onehot[np.argmax(next_gains)] = 1.0
            train_X.append(features)
            train_y.append(onehot)
        return np.array(train_X), np.array(train_y)

    def train_mlp(self, epochs=100, batch_size=32, lr=1e-3):
        train_X, train_y = self.build_trainset()
        X = torch.tensor(train_X, dtype=torch.float32, device=self.device)
        y = torch.tensor(train_y, dtype=torch.float32, device=self.device)
        optimizer = optim.Adam(self.mlp.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=True
        )
        for epoch in range(epochs):
            self.mlp.train()
            total_loss = 0
            for xb, yb in loader:
                optimizer.zero_grad()
                logits = self.mlp(xb)
                # yb is one-hot, convert to class index
                yb_idx = yb.argmax(dim=1)
                loss = criterion(logits, yb_idx)
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * xb.size(0)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataset):.4f}")

    def run_strategy(self, use_mlp=True):
        if use_mlp:
            print("Training MLP for weights optimization...")
            self.train_mlp()
        cur_time = self.earlist_time
        while cur_time <= self.latest_time - self.time_delta:
            if use_mlp:
                hot_coins, _, features = self.pre_trading(cur_time, mlp_infer=False)
                inp = torch.tensor(
                    features, dtype=torch.float32, device=self.device
                ).unsqueeze(0)
                with torch.no_grad():
                    weights = self.mlp(inp).squeeze(0).cpu().numpy()
                self.trading(cur_time, weights=weights)
            else:
                self.trading(cur_time)
            cur_time = self.update_state(cur_time)
        self.analyze()
        self.visualize()


if __name__ == "__main__":
    bt = EnhancedBackTest()
    bt.run_strategy(use_mlp=True)